### 1. Import libraries

In [186]:
import gc
import json
import os
import math
import multiprocessing
import numpy as np
import pandas as pd
import torch
import importlib
import logging
from pathlib import Path
from sklearn.model_selection import GroupKFold, GroupShuffleSplit

# Pycox and PyTorch tuples for survival analysis
import torchtuples as tt
import pycox
from pycox.preprocessing.label_transforms import LabTransDiscreteTime
from pycox.models import CoxPH, DeepHit
from pycox.evaluation import EvalSurv

# Ray for hyperparameter tuning and distributed processing
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.search.optuna import OptunaSearch
from ray.tune.search import ConcurrencyLimiter
from ray.tune.schedulers import ASHAScheduler, PopulationBasedTraining
from ray.air import session
import ray.cloudpickle as pickle

# Custom modules for data handling, balancing, training, evaluation, and model architectures
import dataloader2
import databalancer2
import datatrainer2
import modeleval
import netweaver2

# Reload custom modules to ensure latest changes are available
importlib.reload(dataloader2)
importlib.reload(databalancer2)
importlib.reload(datatrainer2)
importlib.reload(modeleval)
importlib.reload(netweaver2)

# Import specific functions from custom modules to keep code clean and readable
from netweaver2 import (
    lstm_net_init, DHANNWrapper, LSTMWrapper, generalized_ann_net_init
)
from dataloader2 import (
    load_and_transform_data, preprocess_data #stack_sequences, dh_dataset_loader
)
from databalancer2 import (
    define_medoid_general, df_event_focus, rebalance_data, underbalance_data_general, medoid_cluster, 
    dh_rebalance_data
)
from datatrainer2 import (
    recursive_clustering, prepare_training_data, 
    prepare_validation_data, lstm_training
)
from modeleval import (
    dh_test_model, nam_dagostino_chi2, get_baseline_hazard_at_timepoints, combined_test_model
)

import psutil
torch.cuda.empty_cache()
gc.collect()

80

### 2. Define constants, load datasets

In [187]:
# Define Constants and Load Datasets
RANDOM_SEED = 12345
N_SPLIT = 2
FEATURE_COLS = ['gender', 'dm', 'ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3']
DURATION_COL = 'date_from_sub_60'
EVENT_COL = 'endpoint'
CLUSTER_COL = 'key'
TIME_GRID = np.array([i * 365 for i in range(6)])

# Define Feature Groups
CAT_FEATURES = ['gender', 'dm', 'ht', 'sprint']
LOG_FEATURES = ['a1c', 'po4', 'UACR_mg_g', 'Cr']
STANDARD_FEATURES = ['age', 'alb', 'ca', 'hb', 'hco3']
PASSTHROUGH_FEATURES = ['key', 'date_from_sub_60', 'endpoint']

# Load and Transform Data
BASE_FILENAME = '/mnt/d/pydatascience/g3_regress/data/X/X_20240628'
X_train_transformed, X_test_transformed = load_and_transform_data(
    BASE_FILENAME, CAT_FEATURES, LOG_FEATURES, STANDARD_FEATURES, PASSTHROUGH_FEATURES
)

2024-11-21 07:14:59,126 - INFO - Transforming training data...
2024-11-21 07:15:11,857 - INFO - Transforming test data...


### 3. Train and save models
- model naming: {deepsurv/deephit}\_{nn}\_{resample method}_{outcome}
- for deepsurv model, only the result in time_grid will be retrieved so the result of deepsurv and deephit models are compatible

In [188]:
def create_neural_network(config, num_risk = len(X_train_transformed[EVENT_COL].unique()) - 1, num_time_bins=len(TIME_GRID)):
    """
    Function to create a neural network based on the given configuration.

    Args:
        config (dict): Configuration dictionary containing model type, network type, and hyperparameters.

    Returns:
        torch.nn.Module: Created neural network model.
    """
    gc.collect()
    torch.cuda.empty_cache()
    if config['model'] == 'deepsurv':
        num_risk = None
        num_time_bins=None
    elif config['model'] == 'deephit':
        num_risk = num_risk
        num_time_bins = num_time_bins
    # Create the Neural Network
    if config['net'] == 'ann':
        net = generalized_ann_net_init(
            input_size=len(config['features']),
            num_nodes=config["num_nodes"],
            batch_norm=config["batch_norm"],
            dropout=config["dropout"],
            output_size=1, # Default output size for DeepSurv
            num_risks = num_risk,
            num_time_bins = num_time_bins
        )
    elif config['net'] == 'lstm':
        net = lstm_net_init(
            input_size=len(config['features']),
            num_nodes=config["num_nodes"],
            batch_norm=config["batch_norm"],
            dropout=config["dropout"],
            num_risks = num_risk,
            num_time_bins = num_time_bins
        )
    else:
        raise ValueError("Unknown network type: {}".format(config['net']))

    optimizer = tt.optim.AdamWR(decoupled_weight_decay=1e-6, cycle_eta_multiplier=0.8)
    if config['model'] == 'deepsurv':
        model = CoxPH(net, optimizer)
    elif config['model'] == 'deephit':
        model = DeepHit(net, optimizer)
    model.optimizer.set_lr(config["lr"])
    
    return model

def train_neural_network(model, config, X_train, X_val, duration_col, event_col, cluster_col, callbacks, time_grid=None):
    """
    Function to train a given neural network using the provided datasets.

    Args:
        net (torch.nn.Module): Neural network to be trained.
        config (dict): Configuration dictionary containing model hyperparameters.
        X_train (pd.DataFrame): Training dataset with features.
        X_val (pd.DataFrame): Validation dataset with features.
        duration_col (str): Column representing event durations.
        event_col (str): Column representing event occurrences.
        cluster_col (str): Column for grouping during cross-validation.
        callbacks (list): List of callbacks for training.
        time_grid (np.array, optional): Time grid for evaluation if required. Defaults to None.

    Returns:
        model: Trained PyCox model.
        logs: Training logs.
    """
    gc.collect()
    torch.cuda.empty_cache()
    # Train the model
    if config['model'] == 'deepsurv':
        print('Initiate training of deepsurv neural network')
        X_val = df_event_focus(X_val, event_col, config['endpoint'])
        X_val_processed, y_val = preprocess_data(X_val, config['features'], duration_col, event_col)
        val_data = (X_val_processed, y_val)
        if config['net'] == 'ann':
            print('model structure: ANN')
            if config['balance_method'] == 'clustering':
                print('data balancing method: clustering')
                model, logs = recursive_clustering(model, X_train, duration_col, event_col, config, val_data, callbacks, max_repeats=30)
            elif config['balance_method'] == 'enn':
                print('data balancing method: smoteenn')
                X_train = rebalance_data(X_train, event_col, config['endpoint'], CAT_FEATURES, config, RANDOM_SEED, method='ENN')
                X_train, y_train = preprocess_data(X_train, config['features'], duration_col, event_col)
                logs = model.fit(X_train, y_train, config['batch_size'], int(config['max_epochs']), callbacks, verbose=True, val_data=val_data, num_workers=10)
            elif config['balance_method'] == 'tomek':
                print('data balancing method: smotetomek')
                X_train = rebalance_data(X_train, event_col, config['endpoint'], CAT_FEATURES, config, RANDOM_SEED, method='Tomek')
                X_train, y_train = preprocess_data(X_train, config['features'], duration_col, event_col)
                logs = model.fit(X_train, y_train, config['batch_size'], int(config['max_epochs']), callbacks, verbose=True, val_data=val_data, num_workers=10)
        elif config['net'] == 'lstm':
            print('model structure: LSTM')
            if config['balance_method'] == 'clustering':
                print('data balancing method: clustering')
                model, logs = lstm_training(model, X_train, X_val, duration_col, event_col, cluster_col, config, callbacks, time_grid)
            elif config['balance_method'] == 'NearMiss':
                print('data balancing method: NearMiss')
                model, logs = lstm_training(model, X_train, X_val, duration_col, event_col, cluster_col, config, callbacks, time_grid)
    elif config['model'] == 'deephit':
        print('Initiate training of deephit neural network')
        X_val_processed, y_val = preprocess_data(X_val, config['features'], duration_col, event_col, TIME_GRID, discretize=True)
        val_data = (X_val_processed, y_val)
        if config['net'] == 'ann':
            print('model structure: ANN')
            if config['balance_method'] == 'clustering':
                print('data balancing method: clustering')
                model, logs = recursive_clustering(model, X_train, duration_col, event_col, config, val_data, callbacks, max_repeats=30, time_grid=TIME_GRID)
            elif config['balance_method'] == 'NearMiss':
                print('data balancing method: NearMiss')
                X_train = underbalance_data_general(X_train, EVENT_COL, CLUSTER_COL, config, version=config['version'])
                X_train, y_train = preprocess_data(X_train, config['features'], duration_col, event_col, TIME_GRID, discretize=True)
                logs = model.fit(X_train, y_train, config['batch_size'], int(config['max_epochs']), callbacks, verbose=True, val_data=val_data)
        elif config['net'] == 'lstm':
            print('model structure: LSTM')
            if config['balance_method'] == 'clustering':
                print('data balancing method: clustering')
                model, logs = lstm_training(model, X_train, X_val, duration_col, event_col, cluster_col, config, callbacks, time_grid)
            elif config['balance_method'] == 'NearMiss':
                print('data balancing method: NearMiss')
                model, logs = lstm_training(model, X_train, X_val, duration_col, event_col, cluster_col, config, callbacks, time_grid)        

    # Free memory after training
    gc.collect()
    torch.cuda.empty_cache()

    return model, logs

def save_model(params, model, model_path, baseline_hazard_path):
    """
    Save model weights and baseline hazard data.

    Parameters:
    - model: The trained model to save.
    - model_path: Path to save the model weights (.pt file).
    - baseline_hazard_path: Path to save the baseline hazards (.pkl file).
    """
    # Compute baseline hazards and save
    if params['model'] == 'deepsurv':
        baseline_hazard = model.compute_baseline_hazards()
        baseline_hazard.to_pickle(baseline_hazard_path)
    
    # Save model weights
    model.save_model_weights(model_path)
    print(f"Model and baseline hazards saved to {model_path} and {baseline_hazard_path}.")

def training_wrapper(df, config, spliter, model_path, hazard_path, feature_col=FEATURE_COLS, duration_col=DURATION_COL, event_col=EVENT_COL, cluster_col=CLUSTER_COL, time_grid=TIME_GRID):
    """
    Train and save a survival analysis model with grouped cross-validation splits.

    This function performs training on grouped cross-validation splits of the input DataFrame and saves each trained model
    along with its baseline hazards. Memory management is handled to ensure efficient GPU usage.

    Parameters:
    - df (pd.DataFrame): DataFrame containing training data.
    - config (dict): Configuration dictionary for initializing the neural network.
    - spliter (object): Splitter object (e.g., GroupShuffleSplit or StratifiedKFold) used for creating train-validation splits.
    - model_path (str): File path to save the trained model weights (.pt file).
    - hazard_path (str): File path to save the baseline hazards (.pkl file).
    - feature_col (list): List of feature column names in `df` used for model training.
    - duration_col (str): Name of the column representing duration/time-to-event.
    - event_col (str): Name of the column representing the event indicator (0 = censored, 1 = event).
    - cluster_col (str): Name of the column used for grouping (clusters for cross-validation).
    - time_grid (list): List or array defining the time grid for training.

    Returns:
    - None: Saves the model weights and baseline hazard data for each cross-validation split.
    """
    for train_idx, val_idx in spliter.split(X=df[feature_col], y=df[event_col], groups=df[cluster_col]):
        # Clear GPU memory for each split
        gc.collect()
        torch.cuda.empty_cache()
        
        # Define early stopping callback
        callbacks = [tt.cb.EarlyStopping()]
        
        # Create training and validation sets
        train_df = df.iloc[train_idx]
        val_df = df.iloc[val_idx]
        
        # Initialize and train the model
        model = create_neural_network(config)
        model, logs = train_neural_network(
            model, config,
            X_train=train_df, X_val=val_df,
            duration_col=duration_col, event_col=event_col,
            cluster_col=cluster_col, callbacks=callbacks, time_grid=time_grid
        )
        
        # Save the trained model and its baseline hazards
        save_model(config, model, model_path, hazard_path)
        
        # Free memory for the next iteration
        del model, logs
        gc.collect()
        torch.cuda.empty_cache()

    print("Training and saving completed for all cross-validation splits.")

    print("All models have been trained and saved successfully.")

#### 3.1 deepsurv_ann_clustering_1
- features: ['gender', 'dm', 'ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3']
- sampling strategy: 0.05
- 2 hidden layers with 8 and 4 nodes
- no batch normalization in each hidden layer
- dropout ratio in each layer: 0.1144793446270997
- learning rate: 0.1
- max epochs: 9
- batch size: 512

In [189]:
gc.collect()
torch.cuda.empty_cache()

deepsurv_ann_clustering_1_config = {
    'model': 'deepsurv',
    'net': 'ann',
    'balance_method': 'clustering',
    'features': ['gender', 'dm', 'ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3'],
    'endpoint': 1,
    'num_nodes': [8, 4],
    'batch_norm': False,
    'dropout': 0.1144793446270997,
    'lr': 0.1,
    'max_epochs': 9,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 1,
}

#### 3.2 deepsurv_ann_smoteenn_1
- features: ['ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'hb', 'hco3']
- sampling strategy: 0.3
- 4 hidden layers with 64, 32, 16 and 8 nodes
- batch normalization in each hidden layer
- dropout ratio in each layer: 0.09555033386059111
- learning rate: 0.1
- max epochs: 16
- batch size: 512

In [190]:
deepsurv_ann_smoteenn_1_config = {
    'model': 'deepsurv',
    'net': 'ann',
    'balance_method': 'enn',
    'features': ['ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'hb', 'hco3'],
    'endpoint': 1,
    'num_nodes': [64, 32, 16, 8],
    'batch_norm': True,
    'dropout': 0.09555033386059111,
    'lr': 0.1,
    'max_epochs': 16,
    'batch_size': 512,
    'sampling_strategy': 0.3,
    'seq_length': 1,
}

#### 3.3 deepsurv_ann_smotetomek_1
- features:  ['gender', 'dm', 'ht', 'sprint', 'po4', 'UACR_mg_g', 'Cr', 'age', 'hb', 'hco3']
- sampling strategy: 0.2
- 3 hidden layers with 32, 16 and 8 nodes
- batch normalization in each hidden layer
- dropout ratio in each layer: 0.23872991564684112
- learning rate: 0.1
- max epochs: 14
- batch size: 512

In [191]:
deepsurv_ann_smotetomek_1_config = {
    'model': 'deepsurv',
    'net': 'ann',
    'balance_method': 'tomek',
    'features': ['gender', 'dm', 'ht', 'sprint', 'po4', 'UACR_mg_g', 'Cr', 'age', 'hb', 'hco3'],
    'endpoint': 1,
    'num_nodes': [32, 16, 8],
    'batch_norm': True,
    'dropout': 0.23872991564684112,
    'lr': 0.1,
    'max_epochs': 14,
    'batch_size': 512,
    'sampling_strategy': 0.2,
    'seq_length': 1,
}

#### 3.4 deepsurv_ann_clustering_2
- features: ["gender", "a1c", "po4", "UACR_mg_g", "Cr"]
- sampling_strategy: 0.05
- 3 hidden layers with 32, 16, 8 nodes
- no batch normalization in each hidden layer
- dropout ratio in each layer: 0.3058921011568742
- learning rate: 0.1
- max epochs: 14
- batch size: 512

In [192]:
deepsurv_ann_clustering_2_config = {
    'model': 'deepsurv',
    'net': 'ann',
    'balance_method': 'clustering',
    'features': ["gender", "a1c", "po4", "UACR_mg_g", "Cr"],
    'endpoint': 2,
    'num_nodes': [32, 16, 8],
    'batch_norm': False,
    'dropout': 0.3058921011568742,
    'lr': 0.1,
    'max_epochs': 14,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 1,
}

#### 3.5 deepsurv_ann_smoteenn_2
- features: ["gender", "dm", "ht", "sprint", "a1c", "po4", "UACR_mg_g", "Cr", "age", "alb", "ca", "hb", "hco3"]
- sampling_strategy: 0.1, 
- 2 hidden layers with 8, 4 nodes
- no batch normalization in each hidden layer
- dropout ratio in each layer: 0.38878203553667456
- learning rate: 0.01
- max epochs: 10
- batch size: 512

In [193]:
deepsurv_ann_smoteenn_2_config = {
    'model': 'deepsurv',
    'net': 'ann',
    'balance_method': 'enn',
    'features': ["gender", "dm", "ht", "sprint", "a1c", "po4", "UACR_mg_g", "Cr", "age", "alb", "ca", "hb", "hco3"],
    'endpoint': 2,
    'num_nodes': [8, 4],
    'batch_norm': False,
    'dropout': 0.38878203553667456,
    'lr': 0.01,
    'max_epochs': 10,
    'batch_size': 512,
    'sampling_strategy': 0.1,
    'seq_length': 1,
}

#### 3.6 deepsurv_ann_smotetomek_2
- features: ['ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3']
- sampling_strategy: 0.05
- 2 hidden layers with 64, 32 nodes
- batch normalization in each hidden layer 
- dropout ratio in each layer: 0.3162398297390827
- learning rate: 0.1
- max epochs: 11
- batch size: 512

In [194]:
deepsurv_ann_smotetomek_2_config = {
    'model': 'deepsurv',
    'net': 'ann',
    'balance_method': 'tomek',
    'features': ['ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3'],
    'endpoint': 2,
    'num_nodes': [64, 32],
    'batch_norm': True,
    'dropout': 0.3162398297390827,
    'lr': 0.1,
    'max_epochs': 11,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 1,
}

#### 3.7 deepsurv_lstm_clustering_1
- features: ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr']
- sampling_strategy: 0.05
- 3 hidden layers with 8, 4, 2 nodes
- sequence length 7
- no batch normalization in each hidden layer
- dropout ratio in each layer: 0.2772567071863989
- learning rate: 0.1
- max epochs: 13
- batch size: 512

In [195]:
deepsurv_lstm_clustering_1_config = {
    'model': 'deepsurv',
    'net': 'lstm',
    'balance_method': 'clustering',
    'features': ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr'],
    'endpoint': 1,
    'num_nodes': [8, 4, 2],
    'batch_norm': False,
    'dropout': 0.2772567071863989,
    'lr': 0.1,
    'max_epochs': 13,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 7,
}

#### 3.8 deepsurv_lstm_nearmiss_1
- features: ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr']
- sampling_strategy: 0.05
- 3 hidden layers with 8, 4, 2 nodes
- seq_length': 8
- no batch normalization in each hidden layer
- dropout ratio in each layer: 0.3397308077824205
- learning rate: 0.001
- max epochs: 9
- batch size: 512

In [196]:
deepsurv_lstm_nearmiss_1_config = {
    'model': 'deepsurv',
    'net': 'lstm',
    'balance_method': 'NearMiss',
    'features': ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr'],
    'endpoint': 1,
    'num_nodes': [8, 4, 2],
    'batch_norm': False,
    'dropout': 0.3397308077824205,
    'lr': 0.001,
    'max_epochs': 9,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 8,
}

#### 3.9 deepsurv_lstm_clustering_2
- features: ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr']
- sampling_strategy: 0.05
- 3 hidden layers with 8, 4, 2 nodes
- seq_length': 8
- no batch normalization in each hidden layer
- dropout ratio in each layer: 0.3397308077824205
- learning rate: 0.001
- max epochs: 9
- batch size: 512

In [197]:
deepsurv_lstm_clustering_2_config = {
    'model': 'deepsurv',
    'net': 'lstm',
    'balance_method': 'clustering',
    'features': ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr'],
    'endpoint': 2,
    'num_nodes': [8, 4, 2],
    'batch_norm': False,
    'dropout': 0.3397308077824205,
    'lr': 0.001,
    'max_epochs': 9,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 8,
}

#### 3.10 deepsurv_lstm_nearmiss_2
- features: ['sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3']
- sampling_strategy: 0.05
- 2 hidden layers with 32, 16 nodes
- seq_length': 2
- no batch normalization in each hidden layer
- dropout ratio in each layer: 0.35763396978044143
- learning rate: 0.1
- max epochs: 10
- batch size: 512

In [198]:
deepsurv_lstm_nearmiss_2_config = {
    'model': 'deepsurv',
    'net': 'lstm',
    'balance_method': 'NearMiss',
    'features': ['sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3'],
    'endpoint': 2,
    'num_nodes': [32, 16],
    'batch_norm': False,
    'dropout': 0.35763396978044143,
    'lr': 0.1,
    'max_epochs': 10,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 2,
}

#### 3.11 deephit_ann_clustering_all
- features: ['gender', 'dm', 'ht', 'sprint', 'po4', 'UACR_mg_g', 'Cr', 'age', 'hb', 'hco3']
- sampling strategy: 0.05
- 2 hidden layers with 64 and 32 nodes
- batch normalization in each hidden layer
- dropout ratio in each layer: 0.26400151710698067
- learning rate: 0.1
- max epochs: 8
- batch size: 512

In [199]:
deephit_ann_clustering_all_config = {
    'model': 'deephit',
    'net': 'ann',
    'balance_method': 'clustering',
    'features': ['gender', 'dm', 'ht', 'sprint', 'po4', 'UACR_mg_g', 'Cr', 'age', 'hb', 'hco3'],
    'endpoint': 'all',
    'num_nodes': [64, 32],
    'batch_norm': True,
    'dropout': 0.26400151710698067,
    'lr': 0.1,
    'max_epochs': 8,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 1,
}

#### 3.12 deephit_ann_nearmiss2_all
- features: ['sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3']
- sampling strategy: 0.05
- 2 hidden layers with 8, 4 and 2 nodes
- batch normalization in each hidden layer
- dropout ratio in each layer: 0.7346754269827496
- learning rate: 0.01
- max epochs: 7
- batch size: 512

In [200]:
deephit_ann_nearmiss2_all_config = {
    'model': 'deephit',
    'net': 'ann',
    'balance_method': 'NearMiss',
    'version': 2,
    'features': ['sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3'],
    'endpoint': 'all',
    'num_nodes': [8, 4, 2],
    'batch_norm': True,
    'dropout': 0.7346754269827496,
    'lr': 0.01,
    'max_epochs': 7,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 1,
}

#### 3.13 deephit_lstm_clustering_all
- features: ['ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3']
- sampling strategy: 0.05
- seq_length: 6
- 3 hidden layers with 64, 32 and 16 nodes
- batch normalization in each hidden layer
- dropout ratio in each layer: 0.46132889488306583
- learning rate: 0.1
- max epochs: 5
- batch size: 512

In [201]:
deephit_lstm_clustering_all_config = {
    'model': 'deephit',
    'net': 'lstm',
    'balance_method': 'clustering',
    'version': 2,
    'features': ['ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3'],
    'endpoint': 'all',
    'num_nodes': [64, 32, 16],
    'batch_norm': True,
    'dropout': 0.46132889488306583,
    'lr': 0.1,
    'max_epochs': 5,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 6,
}

#### 3.14 deephit_lstm_nearmiss1_all
- features: ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr']
- sampling strategy: 0.05
- seq_length: 9
- 3 hidden layers with 32, 16 and 8 nodes
- batch normalization in each hidden layer
- dropout ratio in each layer: 0.18001924589390816
- learning rate: 0.1
- max epochs: 9
- batch size: 512

In [202]:
deephit_lstm_nearmiss1_all_config = {
    'model': 'deephit',
    'net': 'lstm',
    'balance_method': 'NearMiss',
    'version': 1,
    'features': ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr'],
    'endpoint': 'all',
    'num_nodes': [32, 16, 8],
    'batch_norm': True,
    'dropout': 0.18001924589390816,
    'lr': 0.1,
    'max_epochs': 9,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 9,
}

In [222]:
model_ls = ['deepsurv_ann_clustering_1', 'deepsurv_ann_smoteenn_1', 'deepsurv_ann_smotetomek_1',
            'deepsurv_ann_clustering_2', 'deepsurv_ann_smoteenn_2', 'deepsurv_ann_smotetomek_2',
            'deepsurv_lstm_clustering_1', 'deepsurv_lstm_nearmiss_1', 'deepsurv_lstm_clustering_2', 'deepsurv_lstm_nearmiss_2',
            'deephit_ann_clustering_all', 'deephit_ann_nearmiss2_all', 'deephit_lstm_clustering_all', 'deephit_lstm_nearmiss1_all']
model_path = '/mnt/d/PYDataScience/g3_regress/code/models/'
# Retrieve configurations dynamically from globals
all_configs = {}
for model in model_ls:
    config_var_name = f"{model}_config"
    config = globals().get(config_var_name)
    if config:
        all_configs[model] = config
    else:
        print(f"Configuration for {config_var_name} not found.")

# Save all configurations to a JSON file
output_file = model_path + "all_model_configs.json"
with open(output_file, 'w') as json_file:
    json.dump(all_configs, json_file, indent=4)

print(f"All model configurations saved to {output_file}")

All model configurations saved to /mnt/d/PYDataScience/g3_regress/code/models/all_model_configs.json


In [204]:
gss1 = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_SEED)
gss2 = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_SEED)
for train_idx_1, fin_val_idx in gss1.split(X=X_train_transformed[FEATURE_COLS], y=X_train_transformed[EVENT_COL], groups=X_train_transformed[CLUSTER_COL]):
    X_train_transformed_2, X_fin_val = X_train_transformed.iloc[train_idx_1, :], X_train_transformed.iloc[fin_val_idx, :]
    gc.collect()
    torch.cuda.empty_cache()
    for model in model_ls:
        config_var_name = model + "_config"
        model_config = globals().get(config_var_name)
        if model_config is None:
            print(f"Configuration for {config_var_name} not found.")
            continue

        model_weights_path = f'{model_path}{model}.pt'
        model_hazard_path = f'{model_path}{model}_hazard.pkl'
        
        training_wrapper(X_train_transformed_2, model_config, gss2, model_weights_path, 
                        model_hazard_path, 
                        feature_col=FEATURE_COLS, duration_col=DURATION_COL, event_col=EVENT_COL, cluster_col=CLUSTER_COL, time_grid=TIME_GRID)
        gc.collect()
        torch.cuda.empty_cache()

2024-11-21 07:15:16,843 - INFO - Event column 'endpoint' updated with focus on event value 1.
2024-11-21 07:15:16,848 - INFO - Event column 'endpoint' updated with focus on event value 1.
2024-11-21 07:15:16,856 - INFO - Performing clustering iteration 1 / 20
2024-11-21 07:15:16,856 - INFO - CUDA environment set up and GPU memory cleared.
2024-11-21 07:15:16,859 - INFO - Event column 'endpoint' updated with focus on event value 1.


Initiate training of deepsurv neural network
model structure: ANN
data balancing method: clustering


2024-11-21 07:15:17,464 - INFO - Defined medoid for deepsurv model with 1207 clusters.


0:	[0s / 0s],		train_loss: 5.1261,	val_loss: 7.8751
1:	[0s / 0s],		train_loss: 5.0471,	val_loss: 7.6602
2:	[0s / 0s],		train_loss: 4.9526,	val_loss: 7.5625
3:	[0s / 0s],		train_loss: 4.9109,	val_loss: 7.1527
4:	[0s / 0s],		train_loss: 4.8635,	val_loss: 6.9761
5:	[0s / 0s],		train_loss: 4.8398,	val_loss: 6.9163
6:	[0s / 0s],		train_loss: 4.8417,	val_loss: 6.9144
7:	[0s / 0s],		train_loss: 4.8399,	val_loss: 6.7745
8:	[0s / 0s],		train_loss: 4.8130,	val_loss: 6.8014


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/torchtuples/base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(pat

...

64249


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/torch/nn/modules/rnn.py:917: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:1424.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


0:	[3s / 3s],		train_loss: 0.4252,	val_loss: 0.0750
1:	[2s / 6s],		train_loss: 0.3917,	val_loss: 0.0722
2:	[2s / 9s],		train_loss: 0.3814,	val_loss: 0.0722
3:	[6s / 15s],		train_loss: 0.3792,	val_loss: 0.0669
4:	[2s / 18s],		train_loss: 0.3730,	val_loss: 0.0693
5:	[2s / 21s],		train_loss: 0.3558,	val_loss: 0.0645
6:	[2s / 24s],		train_loss: 0.3377,	val_loss: 0.0655
7:	[3s / 27s],		train_loss: 0.3722,	val_loss: 0.0827
8:	[2s / 30s],		train_loss: 0.3641,	val_loss: 0.0678


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/torchtuples/base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(pat

Model and baseline hazards saved to /mnt/d/PYDataScience/g3_regress/code/models/deephit_lstm_nearmiss1_all.pt and /mnt/d/PYDataScience/g3_regress/code/models/deephit_lstm_nearmiss1_all_hazard.pkl.
Training and saving completed for all cross-validation splits.
All models have been trained and saved successfully.


### 4. Load models and hazards

In [205]:
def load_model(model, model_config, model_path, baseline_hazard_path):
    """
    Load model weights and baseline hazard data.

    Parameters:
    - create_model_func: Function to create the model architecture (e.g., create_neural_network).
    - model_path: Path to load the model weights (.pt file).
    - baseline_hazard_path: Path to load the baseline hazards (.pkl file).

    Returns:
    - model: The loaded model with weights and baseline hazards.
    """
    
    # Load model weights
    model.load_model_weights(model_path)
    
    # Load baseline hazards and assign to model
    if model_config['model'] == 'deepsurv':
        baseline_hazard = pd.read_pickle(baseline_hazard_path)
        model.baseline_hazards_ = baseline_hazard
        model.baseline_cumulative_hazards_ = baseline_hazard.cumsum()
    
    print(f"Model and baseline hazards loaded from {model_path} and {baseline_hazard_path}.")
    return model

In [206]:
# Dictionary to store loaded models
loaded_models = {}

for model_name in model_ls:
    # Retrieve configuration by dynamically constructing the variable name
    config_var_name = model_name + "_config"
    model_config = globals().get(config_var_name)
    
    if model_config is None:
        print(f"Configuration for {config_var_name} not found.")
        continue

    model_weights_path = f'{model_path}{model_name}.pt'
    model_hazard_path = f'{model_path}{model_name}_hazard.pkl'
    
    # Define the model creation function as a lambda to pass the config
    create_model_func = lambda: create_neural_network(
        config=model_config,
        num_risk=len(X_train_transformed[EVENT_COL].unique()) - 1,
        num_time_bins=len(TIME_GRID)
    )
    model = create_model_func()
    
    # Load the model and store it in the dictionary
    loaded_models[model_name] = load_model(model, model_config, model_weights_path, model_hazard_path)
    print(f'Loaded model {model_name}')

Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_clustering_1.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_clustering_1_hazard.pkl.
Loaded model deepsurv_ann_clustering_1


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/torchtuples/base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(pat

Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_smoteenn_1.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_smoteenn_1_hazard.pkl.
Loaded model deepsurv_ann_smoteenn_1
Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_smotetomek_1.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_smotetomek_1_hazard.pkl.
Loaded model deepsurv_ann_smotetomek_1


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/torchtuples/base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(pat

Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_clustering_2.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_clustering_2_hazard.pkl.
Loaded model deepsurv_ann_clustering_2


...

Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_smotetomek_2.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_smotetomek_2_hazard.pkl.
Loaded model deepsurv_ann_smotetomek_2
Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_lstm_clustering_1.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_lstm_clustering_1_hazard.pkl.
Loaded model deepsurv_lstm_clustering_1


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/torchtuples/base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(pat

Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_lstm_nearmiss_1.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_lstm_nearmiss_1_hazard.pkl.
Loaded model deepsurv_lstm_nearmiss_1
Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_lstm_clustering_2.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_lstm_clustering_2_hazard.pkl.
Loaded model deepsurv_lstm_clustering_2


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/torchtuples/base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(pat

Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_lstm_nearmiss_2.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_lstm_nearmiss_2_hazard.pkl.
Loaded model deepsurv_lstm_nearmiss_2
Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deephit_ann_clustering_all.pt and /mnt/d/PYDataScience/g3_regress/code/models/deephit_ann_clustering_all_hazard.pkl.
Loaded model deephit_ann_clustering_all
Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deephit_ann_nearmiss2_all.pt and /mnt/d/PYDataScience/g3_regress/code/models/deephit_ann_nearmiss2_all_hazard.pkl.
Loaded model deephit_ann_nearmiss2_all
Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deephit_lstm_clustering_all.pt and /mnt/d/PYDataScience/g3_regress/code/models/deephit_lstm_clustering_all_hazard.pkl.
Loaded model deephit_lstm_clustering_all
Model and baseline hazards loaded from /mnt/d/

### 5. Prepare and make prediction on the training set, combine the prediction arrays for training of super learner

In [207]:
def predict_neural_network(model, config, X_test, duration_col, event_col, cluster_col, time_grid=None):
    """
    Function to train a given neural network using the provided datasets.

    Args:
        net (torch.nn.Module): Neural network to be trained.
        config (dict): Configuration dictionary containing model hyperparameters.
        X_train (pd.DataFrame): Training dataset with features.
        X_val (pd.DataFrame): Validation dataset with features.
        duration_col (str): Column representing event durations.
        event_col (str): Column representing event occurrences.
        cluster_col (str): Column for grouping during cross-validation.
        callbacks (list): List of callbacks for training.
        time_grid (np.array, optional): Time grid for evaluation if required. Defaults to None.

    Returns:
        model: Trained PyCox model.
        logs: Training logs.
    """
    gc.collect()
    torch.cuda.empty_cache()
    # Train the model
    if config['model'] == 'deepsurv':
        print('Initiate testing of deepsurv neural network')
        X_test = df_event_focus(X_test, event_col, config['endpoint'])
        if config['net'] == 'ann':
            print('model structure: ANN')
            X_test_processed, y_test = preprocess_data(X_test, config['features'], duration_col, event_col)
            surv = model.predict_surv_df(X_test_processed, batch_size=512)
        elif config['net'] == 'lstm':
            print('model structure: LSTM')
            X_test_processed, y_test = prepare_validation_data(X_test, config['features'], duration_col, event_col, config, cluster_col, config['model'], time_grid)
            X_test_tensor = torch.tensor(X_test_processed, dtype=torch.float32)
            y_test_tensor = (torch.tensor(y_test[0], dtype=torch.float32), torch.tensor(y_test[1], dtype=torch.float32))
            surv = model.predict_surv_df(X_test_tensor, batch_size=512)
    elif config['model'] == 'deephit':
        print('Initiate testing of deephit neural network')
        if config['net'] == 'ann':
            print('model structure: ANN')
            X_test_processed, y_test = preprocess_data(X_test, config['features'], duration_col, event_col, time_grid, discretize=True)
            surv = model.predict_cif(X_test_processed, batch_size=512)
            print('prediction complete, please note that prediction of deephit models are CIF.')
        elif config['net'] == 'lstm':
            print('model structure: LSTM')
            X_test_processed, y_test = prepare_validation_data(X_test, config['features'], duration_col, event_col, config, cluster_col, config['model'], time_grid)
            surv = model.predict_cif(X_test_processed, batch_size=512)
            print('prediction complete, please note that prediction of deephit models are CIF.')

    # Free memory after training
    gc.collect()
    torch.cuda.empty_cache()

    return surv, y_test

In [208]:

def align_to_time_grid(surv, time_grid):
    """
    Align the survival DataFrame to the closest indices of the time grid.

    Parameters:
        surv (pd.DataFrame): Survival probabilities DataFrame.
        time_grid (np.array): Array of target time points to align.

    Returns:
        aligned_surv (pd.DataFrame): Aligned survival probabilities.
    """
    # Convert the DataFrame's index to a NumPy array for fast computation
    surv_times = np.array(surv.index)
    
    # Find the closest time in the survival DataFrame for each time in the grid
    closest_indices = [np.argmin(np.abs(surv_times - t)) for t in time_grid]
    
    # Extract the rows corresponding to the closest times
    aligned_surv = surv.iloc[closest_indices].copy()
    
    # Reindex the DataFrame to match the time grid
    aligned_surv.index = range(len(time_grid))  # Standardize indices to 0, 1, 2, ...
    return aligned_surv

#### 5.1 Use lifelines and CoxPH Fitter to get the CIF of both outcomes as the 'ground truth' of the training dataset

In [209]:
from lifelines import CoxPHFitter
from lifelines import AalenJohansenFitter
from sklearn.linear_model import LinearRegression
from joblib import Parallel, delayed

# Convert all non-target events to 0 (censored)
X_train_transformed["event1"] = X_train_transformed[EVENT_COL].apply(lambda x: 1 if x == 1 else 0)
X_train_transformed["event2"] = X_train_transformed[EVENT_COL].apply(lambda x: 1 if x == 2 else 0)

class_counts = X_train_transformed[EVENT_COL].value_counts()
X_train_transformed['weights'] = X_train_transformed[EVENT_COL].map(lambda e: 1 / class_counts[e]).values

# Step 1: Fit a Cox model for each event type
cox_model_event_1 = CoxPHFitter()
cox_model_event_1.fit(X_train_transformed[FEATURE_COLS + [DURATION_COL, 'event1', CLUSTER_COL, 'weights']], duration_col=DURATION_COL, event_col="event1", cluster_col=CLUSTER_COL, weights_col="weights", robust=True)

cox_model_event_2 = CoxPHFitter()
cox_model_event_2.fit(X_train_transformed[FEATURE_COLS + [DURATION_COL, 'event2', CLUSTER_COL, 'weights']], duration_col=DURATION_COL, event_col="event2", cluster_col=CLUSTER_COL, weights_col="weights", robust=True)

# Step 2: Predict individual cumulative hazards for each event type
cumulative_hazard_event_1 = cox_model_event_1.predict_cumulative_hazard(X_train_transformed)
cumulative_hazard_event_2 = cox_model_event_2.predict_cumulative_hazard(X_train_transformed)

# Step 3: Compute overall survival for each individual
# Overall survival: S(t) = exp(- (H1(t) + H2(t)))
overall_survival = np.exp(-(cumulative_hazard_event_1 + cumulative_hazard_event_2))

# Step 4: Calculate CIF for each event type
# CIF_k(t) = ∫ h_k(u) * S(u) du (approximated as cumulative sum)
cif_event_1 = (cumulative_hazard_event_1 * overall_survival).cumsum(axis=0)
cif_event_2 = (cumulative_hazard_event_2 * overall_survival).cumsum(axis=0)

# Step 5: Format and display the CIF predictions
cif_event_1_normalized = cif_event_1.div(cif_event_1.iloc[-1].max(), axis=1)
cif_event_2_normalized = cif_event_2.div(cif_event_2.iloc[-1].max(), axis=1)

# Compute the CIF ground truth
cif_ground_truth = np.zeros((2,6,396424))
cif_ground_truth[0] = align_to_time_grid(cif_event_1_normalized, TIME_GRID).values
cif_ground_truth[1] = align_to_time_grid(cif_event_2_normalized, TIME_GRID).values

/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/coxph_fitter.py:1607: ConvergenceWarning: Newton-Raphson convergence completed successfully but norm(delta) is still high, 0.359. This may imply non-unique solutions to the maximum likelihood. Perhaps there is collinearity or complete separation in the dataset?

  warnings.warn(


In [220]:
_, y_ground_truth = preprocess_data(X_train_transformed, FEATURE_COLS, DURATION_COL, EVENT_COL, TIME_GRID, discretize=True)
concordance_indices = {}
integrated_brier_scores = {}
neg_log_likelihoods = {}
brier_series = {}
for i in range(0, 2):
    event_interest = i + 1
    cif = pd.DataFrame(cif_ground_truth[i], [0, 1, 2, 3, 4, 5])
    ev = EvalSurv(1-cif, y_ground_truth[0], y_ground_truth[1] == event_interest, censor_surv='km')
    concordance_indices[f"Event_{event_interest}"] = ev.concordance_td()
    brier_series[f"Event_{event_interest}"] = ev.brier_score(TIME_GRID)
    integrated_brier_scores[f"Event_{event_interest}"] = ev.integrated_brier_score(TIME_GRID)
    neg_log_likelihoods[f"Event_{event_interest}"] = ev.integrated_nbll(TIME_GRID)

    # # Nam and D'Agostino Chi2 statistic for calibration
    # for time in time_grid:
    #     chi2_stat, p_value, observed_events, expected_events, n, prob_df = nam_dagostino_chi2(
    #         df=df_test, 
    #         duration_col=duration_col, 
    #         event_col=event_col,
    #         surv=(1-cif), 
    #         time=time, 
    #         event_focus=event_interest
    #     )
    #     nam_dagostino_results.append({
    #         'Event': event_interest,
    #         'Year': round(time / 365),
    #         'Chi2_Stat': chi2_stat,
    #         'P_Value': p_value,
    #         'Observed_Events': observed_events.tolist(),
    #         'Expected_Events': expected_events.tolist(),
    #         'Sample_Size': n.tolist()
    #     })
display(concordance_indices)
display(brier_series)
display(integrated_brier_scores)
display(neg_log_likelihoods)


{'Event_1': 0.9818982092210652, 'Event_2': 0.7033707118427875}

{'Event_1': 0       0.000000
 365     0.038203
 730     0.038203
 1095    0.038203
 1460    0.038203
 1825    0.038203
 Name: brier_score, dtype: float64,
 'Event_2': 0       0.000000
 365     0.085103
 730     0.085103
 1095    0.085103
 1460    0.085103
 1825    0.085103
 Name: brier_score, dtype: float64}

{'Event_1': 0.03565589439044669, 'Event_2': 0.07942951892202992}

{'Event_1': 0.17732477425591783, 'Event_2': 0.27434970836708145}

#### 5.2 Get the prediction for each model

In [211]:
gc.collect()
torch.cuda.empty_cache()

# Initialize dictionary to store combined predictions and y_test
combined_predictions = []
combined_y_test = []
cif_ground_truth_test = []

# Get unique keys and split them into 10 groups
unique_keys = X_train_transformed_2['key'].unique()
np.random.shuffle(unique_keys)  # Shuffle keys to ensure randomness
key_folds = np.array_split(unique_keys, 10)

for fold_idx, test_keys in enumerate(key_folds):
    print(f"Processing fold {fold_idx + 1}...")
    # Split the dataset into train and test based on keys
    X_test_fold = X_train_transformed_2[X_train_transformed_2['key'].isin(test_keys)]
    
    # Get the indices of X_test_fold relative to the original dataset
    test_indices = X_test_fold.index.to_numpy()

    # Extract CIF ground truth for these indices
    fold_cif_ground_truth = cif_ground_truth[:, :, test_indices]

    # Stack this fold's CIF ground truth
    cif_ground_truth_test.append(fold_cif_ground_truth)
    
    # Store predictions for this fold
    fold_predictions = {}
    
    X_test, y_test = preprocess_data(X_test_fold, FEATURE_COLS, DURATION_COL, EVENT_COL, TIME_GRID, discretize=True)
    combined_y_test.append(y_test)
    
    for model_name in model_ls:
        # Retrieve configuration by dynamically constructing the variable name
        config_var_name = model_name + "_config"
        model_config = globals().get(config_var_name)
        if model_config is None:
            print(f"Configuration for {config_var_name} not found.")
            continue
        try:
            print(f"Initiating prediction for model: {model_name}")
            
            # Retrieve the loaded model
            model = loaded_models.get(model_name)
            if model is None:
                print(f"Model {model_name} is not loaded.")
                continue
            
            # Predict using the loaded model and configuration
            surv, _ = predict_neural_network(
                model=model,
                config=model_config,
                X_test=X_test_fold,
                duration_col=DURATION_COL,
                event_col=EVENT_COL,
                cluster_col=CLUSTER_COL,
                time_grid=TIME_GRID
            )
            
            # Align survival probabilities (if DeepSurv)
            if model_config['model'] == 'deepsurv':
                surv = align_to_time_grid(surv, TIME_GRID).values  # 2D array
                
                # Structure key dynamically
                key = f"deepsurv_{model_config['net']}_{model_config['balance_method']}"
                
                # Store predictions
                if key not in fold_predictions:
                    fold_predictions[key] = []
                fold_predictions[key].append(np.expand_dims(1 - surv, axis=0))
            
            # Handle DeepHit predictions
            elif model_config['model'] == 'deephit':
                surv = np.array(surv)  # Convert to numpy array
                
                # Structure key dynamically
                key = f"deephit_{model_config['net']}_{model_config['balance_method']}"
                
                # Store predictions
                if key not in fold_predictions:
                    fold_predictions[key] = []
                fold_predictions[key].append(surv)
            
            print(f"Prediction completed for {model_name} on fold {fold_idx + 1}.")
        
        except Exception as e:
            print(f"Error during prediction for {model_name} on fold {fold_idx + 1}: {e}")
            
    combined_predictions.append(fold_predictions)

meta_learner_X_train = {}

for fold_predictions in combined_predictions:
    for model_key, fold_data in fold_predictions.items():
        if model_key not in meta_learner_X_train:
            meta_learner_X_train[model_key] = []
        
        # Align DeepSurv predictions to match DeepHit format
        if "deepsurv" in model_key:
            # Stack competing outcomes and remove the redundant dimension
            meta_learner_X_train[model_key].extend(
                [np.squeeze(np.stack(fold_data, axis=0), axis=1)]  # Squeeze out extra axis
            )
        else:
            # Keep DeepHit predictions as-is
            meta_learner_X_train[model_key].extend(fold_data)

# Combine y_test
meta_learner_y_train = (
    np.concatenate([fold[0] for fold in combined_y_test]),  # Concatenate all first elements
    np.concatenate([fold[1] for fold in combined_y_test])   # Concatenate all second elements
)

# Combine CIF ground truth for all folds
cif_ground_truth_test_stacked = np.concatenate(cif_ground_truth_test, axis=2)

print("Final predictions and y_test combined.")

2024-11-21 09:05:26,695 - INFO - Event column 'endpoint' updated with focus on event value 1.


Processing fold 1...
Initiating prediction for model: deepsurv_ann_clustering_1
Initiate testing of deepsurv neural network
model structure: ANN


2024-11-21 09:05:27,340 - INFO - Event column 'endpoint' updated with focus on event value 1.


Prediction completed for deepsurv_ann_clustering_1 on fold 1.
Initiating prediction for model: deepsurv_ann_smoteenn_1
Initiate testing of deepsurv neural network
model structure: ANN


2024-11-21 09:05:36,046 - INFO - Event column 'endpoint' updated with focus on event value 1.


...

Prediction completed for deepsurv_lstm_nearmiss_2 on fold 10.
Initiating prediction for model: deephit_ann_clustering_all
Initiate testing of deephit neural network
model structure: ANN
prediction complete, please note that prediction of deephit models are CIF.
Prediction completed for deephit_ann_clustering_all on fold 10.
Initiating prediction for model: deephit_ann_nearmiss2_all
Initiate testing of deephit neural network
model structure: ANN
prediction complete, please note that prediction of deephit models are CIF.
Prediction completed for deephit_ann_nearmiss2_all on fold 10.
Initiating prediction for model: deephit_lstm_clustering_all
Initiate testing of deephit neural network
model structure: LSTM


2024-11-21 09:22:40,206 - INFO - Validation data retrieved


prediction complete, please note that prediction of deephit models are CIF.
Prediction completed for deephit_lstm_clustering_all on fold 10.
Initiating prediction for model: deephit_lstm_nearmiss1_all
Initiate testing of deephit neural network
model structure: LSTM


2024-11-21 09:22:53,764 - INFO - Validation data retrieved


prediction complete, please note that prediction of deephit models are CIF.
Prediction completed for deephit_lstm_nearmiss1_all on fold 10.
Final predictions and y_test combined.


In [212]:
# Combine all 10 items for each model's predictions
final_meta_learner_X_train = {}

for key, predictions in meta_learner_X_train.items():
    # Concatenate the predictions along the last axis
    final_meta_learner_X_train[key] = np.concatenate(predictions, axis=2)  # Combine along the feature axis

# Validate the shapes
for key, combined_prediction in final_meta_learner_X_train.items():
    print(f"{key}: final shape {combined_prediction.shape}")


deepsurv_ann_clustering: final shape (2, 6, 316242)
deepsurv_ann_enn: final shape (2, 6, 316242)
deepsurv_ann_tomek: final shape (2, 6, 316242)
deepsurv_lstm_clustering: final shape (2, 6, 316242)
deepsurv_lstm_NearMiss: final shape (2, 6, 316242)
deephit_ann_clustering: final shape (2, 6, 316242)
deephit_ann_NearMiss: final shape (2, 6, 316242)
deephit_lstm_clustering: final shape (2, 6, 316242)
deephit_lstm_NearMiss: final shape (2, 6, 316242)


In [213]:
display(np.shape(cif_ground_truth_test_stacked))

# Use the duration and event data as targets
durations = meta_learner_y_train[0]  # Time to event or censoring
events = meta_learner_y_train[1]  # Event type (competing risks)

print(durations.shape)
print(events.shape)
print(np.unique(events, return_counts=True))

(2, 6, 316242)

(316242,)
(316242,)
(array([0, 1, 2]), array([310074,   1509,   4659]))


### 6. Ensemble methods

#### 6.1 Prepare each model's prediction on validation set


In [214]:
gc.collect()
torch.cuda.empty_cache()

# Initialize dictionary to store combined predictions and y_test
combined_predictions = []
combined_y_val = []
cif_ground_truth_val = []

# Get unique keys and split them into 10 groups
unique_keys = X_fin_val['key'].unique()
np.random.shuffle(unique_keys)  # Shuffle keys to ensure randomness
key_folds = np.array_split(unique_keys, 10)

for fold_idx, test_keys in enumerate(key_folds):
    print(f"Processing fold {fold_idx + 1}...")
    # Split the dataset into train and test based on keys
    X_val_fold = X_fin_val[X_fin_val['key'].isin(test_keys)]
    
    # Get the indices of X_test_fold relative to the original dataset
    test_indices = X_val_fold.index.to_numpy()

    # Extract CIF ground truth for these indices
    cif_ground_truth_val_fold = cif_ground_truth[:, :, test_indices]

    # Stack this fold's CIF ground truth
    cif_ground_truth_val.append(cif_ground_truth_val_fold)
    
    # Store predictions for this fold
    fold_predictions = {}
    
    X_val, y_val = preprocess_data(X_val_fold, FEATURE_COLS, DURATION_COL, EVENT_COL, TIME_GRID, discretize=True)
    combined_y_val.append(y_val)
    
    for model_name in model_ls:
        # Retrieve configuration by dynamically constructing the variable name
        config_var_name = model_name + "_config"
        model_config = globals().get(config_var_name)
        if model_config is None:
            print(f"Configuration for {config_var_name} not found.")
            continue
        try:
            print(f"Initiating prediction for model: {model_name}")
            
            # Retrieve the loaded model
            model = loaded_models.get(model_name)
            if model is None:
                print(f"Model {model_name} is not loaded.")
                continue
            
            # Predict using the loaded model and configuration
            surv, _ = predict_neural_network(
                model=model,
                config=model_config,
                X_test=X_val_fold,
                duration_col=DURATION_COL,
                event_col=EVENT_COL,
                cluster_col=CLUSTER_COL,
                time_grid=TIME_GRID
            )
            
            # Align survival probabilities (if DeepSurv)
            if model_config['model'] == 'deepsurv':
                surv = align_to_time_grid(surv, TIME_GRID).values  # 2D array
                
                # Structure key dynamically
                key = f"deepsurv_{model_config['net']}_{model_config['balance_method']}"
                
                # Store predictions
                if key not in fold_predictions:
                    fold_predictions[key] = []
                fold_predictions[key].append(np.expand_dims(1 - surv, axis=0))
            
            # Handle DeepHit predictions
            elif model_config['model'] == 'deephit':
                surv = np.array(surv)  # Convert to numpy array
                
                # Structure key dynamically
                key = f"deephit_{model_config['net']}_{model_config['balance_method']}"
                
                # Store predictions
                if key not in fold_predictions:
                    fold_predictions[key] = []
                fold_predictions[key].append(surv)
            
            print(f"Prediction completed for {model_name} on fold {fold_idx + 1}.")
        
        except Exception as e:
            print(f"Error during prediction for {model_name} on fold {fold_idx + 1}: {e}")
            
    combined_predictions.append(fold_predictions)

meta_learner_X_val = {}

for fold_predictions in combined_predictions:
    for model_key, fold_data in fold_predictions.items():
        if model_key not in meta_learner_X_val:
            meta_learner_X_val[model_key] = []
        
        # Align DeepSurv predictions to match DeepHit format
        if "deepsurv" in model_key:
            # Stack competing outcomes and remove the redundant dimension
            meta_learner_X_val[model_key].extend(
                [np.squeeze(np.stack(fold_data, axis=0), axis=1)]  # Squeeze out extra axis
            )
        else:
            # Keep DeepHit predictions as-is
            meta_learner_X_val[model_key].extend(fold_data)

# Combine y_test
meta_learner_y_val = (
    np.concatenate([fold[0] for fold in combined_y_val]),  # Concatenate all first elements
    np.concatenate([fold[1] for fold in combined_y_val])   # Concatenate all second elements
)

# Combine CIF ground truth for all folds
cif_ground_truth_val_stacked = np.concatenate(cif_ground_truth_val, axis=2)

print("Validation predictions and y_val combined.")

Processing fold 1...
Initiating prediction for model: deepsurv_ann_clustering_1
Initiate testing of deepsurv neural network


2024-11-21 09:22:54,543 - INFO - Event column 'endpoint' updated with focus on event value 1.


model structure: ANN


2024-11-21 09:22:55,018 - INFO - Event column 'endpoint' updated with focus on event value 1.


Prediction completed for deepsurv_ann_clustering_1 on fold 1.
Initiating prediction for model: deepsurv_ann_smoteenn_1
Initiate testing of deepsurv neural network
model structure: ANN


...

Prediction completed for deepsurv_lstm_nearmiss_2 on fold 10.
Initiating prediction for model: deephit_ann_clustering_all
Initiate testing of deephit neural network
model structure: ANN
prediction complete, please note that prediction of deephit models are CIF.
Prediction completed for deephit_ann_clustering_all on fold 10.
Initiating prediction for model: deephit_ann_nearmiss2_all
Initiate testing of deephit neural network
model structure: ANN
prediction complete, please note that prediction of deephit models are CIF.
Prediction completed for deephit_ann_nearmiss2_all on fold 10.
Initiating prediction for model: deephit_lstm_clustering_all
Initiate testing of deephit neural network
model structure: LSTM


2024-11-21 09:27:27,644 - INFO - Validation data retrieved


prediction complete, please note that prediction of deephit models are CIF.
Prediction completed for deephit_lstm_clustering_all on fold 10.
Initiating prediction for model: deephit_lstm_nearmiss1_all
Initiate testing of deephit neural network
model structure: LSTM


2024-11-21 09:27:30,129 - INFO - Validation data retrieved


prediction complete, please note that prediction of deephit models are CIF.
Prediction completed for deephit_lstm_nearmiss1_all on fold 10.
Validation predictions and y_val combined.


In [215]:
# Combine all 10 items for each model's predictions
final_meta_learner_X_val = {}

for key, predictions in meta_learner_X_val.items():
    # Concatenate the predictions along the last axis
    final_meta_learner_X_val[key] = np.concatenate(predictions, axis=2)  # Combine along the feature axis

# Validate the shapes
for key, combined_prediction in final_meta_learner_X_val.items():
    print(f"{key}: final shape {combined_prediction.shape}")

deepsurv_ann_clustering: final shape (2, 6, 80182)
deepsurv_ann_enn: final shape (2, 6, 80182)
deepsurv_ann_tomek: final shape (2, 6, 80182)
deepsurv_lstm_clustering: final shape (2, 6, 80182)
deepsurv_lstm_NearMiss: final shape (2, 6, 80182)
deephit_ann_clustering: final shape (2, 6, 80182)
deephit_ann_NearMiss: final shape (2, 6, 80182)
deephit_lstm_clustering: final shape (2, 6, 80182)
deephit_lstm_NearMiss: final shape (2, 6, 80182)


In [216]:
# Use the duration and event data as targets
durations_val = meta_learner_y_val[0]  # Time to event or censoring
events_val = meta_learner_y_val[1]  # Event type (competing risks)

print(durations_val.shape)
print(events_val.shape)
print(np.unique(events_val, return_counts=True))

(80182,)
(80182,)
(array([0, 1, 2]), array([78593,   416,  1173]))


#### 6.2 Prepare each model's prediction on test set

In [217]:
gc.collect()
torch.cuda.empty_cache()

# Initialize dictionary to store combined predictions and y_test
combined_predictions = []
combined_y_fin_test = []
cif_ground_truth_fin_test = []

# Get unique keys and split them into 10 groups
unique_keys = X_test_transformed['key'].unique()
np.random.shuffle(unique_keys)  # Shuffle keys to ensure randomness
key_folds = np.array_split(unique_keys, 10)

for fold_idx, test_keys in enumerate(key_folds):
    print(f"Processing fold {fold_idx + 1}...")
    # Split the dataset into train and test based on keys
    X_test_fold = X_test_transformed[X_test_transformed['key'].isin(test_keys)]
    
    # Get the indices of X_test_fold relative to the original dataset
    test_indices = X_test_transformed.index.to_numpy()
    
    # Store predictions for this fold
    fold_predictions = {}
    
    X_fin_test, y_fin_test = preprocess_data(X_test_fold, FEATURE_COLS, DURATION_COL, EVENT_COL, TIME_GRID, discretize=True)
    combined_y_fin_test.append(y_fin_test)
    
    for model_name in model_ls:
        # Retrieve configuration by dynamically constructing the variable name
        config_var_name = model_name + "_config"
        model_config = globals().get(config_var_name)
        if model_config is None:
            print(f"Configuration for {config_var_name} not found.")
            continue
        try:
            print(f"Initiating prediction for model: {model_name}")
            
            # Retrieve the loaded model
            model = loaded_models.get(model_name)
            if model is None:
                print(f"Model {model_name} is not loaded.")
                continue
            
            # Predict using the loaded model and configuration
            surv, _ = predict_neural_network(
                model=model,
                config=model_config,
                X_test=X_test_fold,
                duration_col=DURATION_COL,
                event_col=EVENT_COL,
                cluster_col=CLUSTER_COL,
                time_grid=TIME_GRID
            )
            
            # Align survival probabilities (if DeepSurv)
            if model_config['model'] == 'deepsurv':
                surv = align_to_time_grid(surv, TIME_GRID).values  # 2D array
                
                # Structure key dynamically
                key = f"deepsurv_{model_config['net']}_{model_config['balance_method']}"
                
                # Store predictions
                if key not in fold_predictions:
                    fold_predictions[key] = []
                fold_predictions[key].append(np.expand_dims(1 - surv, axis=0))
            
            # Handle DeepHit predictions
            elif model_config['model'] == 'deephit':
                surv = np.array(surv)  # Convert to numpy array
                
                # Structure key dynamically
                key = f"deephit_{model_config['net']}_{model_config['balance_method']}"
                
                # Store predictions
                if key not in fold_predictions:
                    fold_predictions[key] = []
                fold_predictions[key].append(surv)
            
            print(f"Prediction completed for {model_name} on fold {fold_idx + 1}.")
        
        except Exception as e:
            print(f"Error during prediction for {model_name} on fold {fold_idx + 1}: {e}")
            
    combined_predictions.append(fold_predictions)

meta_learner_X_fin_test = {}

for fold_predictions in combined_predictions:
    for model_key, fold_data in fold_predictions.items():
        if model_key not in meta_learner_X_fin_test:
            meta_learner_X_fin_test[model_key] = []
        
        # Align DeepSurv predictions to match DeepHit format
        if "deepsurv" in model_key:
            # Stack competing outcomes and remove the redundant dimension
            meta_learner_X_fin_test[model_key].extend(
                [np.squeeze(np.stack(fold_data, axis=0), axis=1)]  # Squeeze out extra axis
            )
        else:
            # Keep DeepHit predictions as-is
            meta_learner_X_fin_test[model_key].extend(fold_data)

# Combine y_test
meta_learner_y_fin_test = (
    np.concatenate([fold[0] for fold in combined_y_fin_test]),  # Concatenate all first elements
    np.concatenate([fold[1] for fold in combined_y_fin_test])   # Concatenate all second elements
)

print("Final test predictions and y_fin_test combined.")

Processing fold 1...
Initiating prediction for model: deepsurv_ann_clustering_1
Initiate testing of deepsurv neural network


2024-11-21 09:27:30,742 - INFO - Event column 'endpoint' updated with focus on event value 1.


model structure: ANN


2024-11-21 09:27:31,161 - INFO - Event column 'endpoint' updated with focus on event value 1.


Prediction completed for deepsurv_ann_clustering_1 on fold 1.
Initiating prediction for model: deepsurv_ann_smoteenn_1
Initiate testing of deepsurv neural network
model structure: ANN


...

Prediction completed for deepsurv_lstm_nearmiss_2 on fold 10.
Initiating prediction for model: deephit_ann_clustering_all
Initiate testing of deephit neural network
model structure: ANN
prediction complete, please note that prediction of deephit models are CIF.
Prediction completed for deephit_ann_clustering_all on fold 10.
Initiating prediction for model: deephit_ann_nearmiss2_all
Initiate testing of deephit neural network
model structure: ANN
prediction complete, please note that prediction of deephit models are CIF.
Prediction completed for deephit_ann_nearmiss2_all on fold 10.
Initiating prediction for model: deephit_lstm_clustering_all
Initiate testing of deephit neural network
model structure: LSTM


2024-11-21 09:30:14,746 - INFO - Validation data retrieved


prediction complete, please note that prediction of deephit models are CIF.
Prediction completed for deephit_lstm_clustering_all on fold 10.
Initiating prediction for model: deephit_lstm_nearmiss1_all
Initiate testing of deephit neural network
model structure: LSTM


2024-11-21 09:30:16,095 - INFO - Validation data retrieved


prediction complete, please note that prediction of deephit models are CIF.
Prediction completed for deephit_lstm_nearmiss1_all on fold 10.
Final test predictions and y_fin_test combined.


In [218]:
# Combine all 10 items for each model's predictions
final_meta_learner_X_fin_test = {}

for key, predictions in meta_learner_X_fin_test.items():
    # Concatenate the predictions along the last axis
    final_meta_learner_X_fin_test[key] = np.concatenate(predictions, axis=2)  # Combine along the feature axis

# Validate the shapes
for key, combined_prediction in final_meta_learner_X_fin_test.items():
    print(f"{key}: final shape {combined_prediction.shape}")
    
# Use the duration and event data as targets
durations_fin_test = meta_learner_y_fin_test[0]  # Time to event or censoring
events_fin_test = meta_learner_y_fin_test[1]  # Event type (competing risks)

print(durations_fin_test.shape)
print(events_fin_test.shape)
print(np.unique(events_fin_test, return_counts=True))

deepsurv_ann_clustering: final shape (2, 6, 40513)
deepsurv_ann_enn: final shape (2, 6, 40513)
deepsurv_ann_tomek: final shape (2, 6, 40513)
deepsurv_lstm_clustering: final shape (2, 6, 40513)
deepsurv_lstm_NearMiss: final shape (2, 6, 40513)
deephit_ann_clustering: final shape (2, 6, 40513)
deephit_ann_NearMiss: final shape (2, 6, 40513)
deephit_lstm_clustering: final shape (2, 6, 40513)
deephit_lstm_NearMiss: final shape (2, 6, 40513)
(40513,)
(40513,)
(array([0, 1, 2]), array([39793,   200,   520]))


#### 6.2 Averaging

In [219]:
# Predictions from each model
predictions = [final_meta_learner_X_fin_test[key] for key in final_meta_learner_X_fin_test.keys()]

# Aggregate by majority voting for each time point and event type
average_predictions = np.mean(predictions, axis=0)

print("Average Voting Predictions shape:", average_predictions.shape)


Average Voting Predictions shape: (2, 6, 40513)


In [ ]:
concordance_indices = {}
integrated_brier_scores = {}
neg_log_likelihoods = {}
brier_series = {}
nam_dagostino_results = []  # Initialize as a list

for i in range(0, 2):
    event_interest = i + 1
    cif = pd.DataFrame(average_predictions[i], index=[0, 1, 2, 3, 4, 5])  # Use index correctly
    ev = EvalSurv(1 - cif, durations_fin_test, events_fin_test == event_interest, censor_surv='km')

    # Concordance index
    concordance_indices[f"Event_{event_interest}"] = ev.concordance_td()

    # Brier score series
    brier_series[f"Event_{event_interest}"] = ev.brier_score(TIME_GRID)

    # Integrated Brier score
    integrated_brier_scores[f"Event_{event_interest}"] = ev.integrated_brier_score(TIME_GRID)

    # Negative log-likelihood
    neg_log_likelihoods[f"Event_{event_interest}"] = ev.integrated_nbll(TIME_GRID)

    # Nam and D'Agostino Chi2 statistic for calibration
    for time_idx, time in enumerate(TIME_GRID):
        # Filter data for current time point
        mask = durations_fin_test <= time
        durations_filtered = durations_fin_test[mask]
        events_filtered = events_fin_test[mask]
        
        if len(durations_filtered) == 0:
            print(f"Skipping time {time} for event {event_interest} due to empty data.")
            continue
        
        try:
            # Calculate Nam-D'Agostino Chi² statistic
            chi2_stat, p_value, observed_events, expected_events, n, prob_df = nam_dagostino_chi2(
                df=pd.DataFrame({"durations": durations_filtered, "events": events_filtered}),
                duration_col="durations",
                event_col="events",
                surv=(1 - cif),  # Survival function
                time=time_idx,
                event_focus=event_interest
            )

            # Append results
            nam_dagostino_results.append({
                'Event': event_interest,
                'Year': round(time / 365),
                'Chi2_Stat': chi2_stat,
                'P_Value': p_value,
                'Observed_Events': observed_events.tolist() if isinstance(observed_events, np.ndarray) else observed_events,
                'Expected_Events': expected_events.tolist() if isinstance(expected_events, np.ndarray) else expected_events,
                'Sample_Size': n if isinstance(n, int) else n.tolist()
            })
        except ValueError as e:
            print(f"Error calculating Nam-D'Agostino Chi² at time {time} for event {event_interest}: {e}")

# Display results
print("Concordance Indices:", concordance_indices)
print("Brier Score Series:", brier_series)
print("Integrated Brier Scores:", integrated_brier_scores)
print("Negative Log-Likelihoods:", neg_log_likelihoods)
print("Nam-D'Agostino Results:", nam_dagostino_results)



Error calculating Nam-D'Agostino Chi² at time 0 for event 1: Length of values (40513) does not match length of index (19588)
Error calculating Nam-D'Agostino Chi² at time 0 for event 2: Length of values (40513) does not match length of index (19588)


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event

Concordance Indices: {'Event_1': 0.9861141445942802, 'Event_2': 0.8047894453585371}
Brier Score Series: {'Event_1': 0       3.783331e-07
365     4.689445e-02
730     4.689445e-02
1095    4.689445e-02
1460    4.689445e-02
1825    4.689445e-02
Name: brier_score, dtype: float64, 'Event_2': 0       6.123067e-07
365     2.107838e-01
730     2.107838e-01
1095    2.107838e-01
1460    2.107838e-01
1825    2.107838e-01
Name: brier_score, dtype: float64}
Integrated Brier Scores: {'Event_1': 0.04376818205145606, 'Event_2': 0.19673160328047123}
Negative Log-Likelihoods: {'Event_1': 0.22729299286338747, 'Event_2': 0.5733443844258161}
Nam-D'Agostino Results: [{'Event': 1, 'Year': 1, 'Chi2_Stat': 0.12214437522851679, 'P_Value': 0.9982093122377609, 'Observed_Events': quantile
0    0.000000
1    0.000000
2    0.000000
3    0.000000
4    0.022289
Name: observed_probs, dtype: float64, 'Expected_Events': quantile
0    0.016495
1    0.018913
2    0.023356
3    0.026458
4    0.074863
Name: predicted_probs, 

/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(


In [ ]:
np.unique(durations_fin_test)

array([0, 1, 2, 3, 4, 5])

#### 6.3 Stacking

##### 6.3.1 XGBoost

In [ ]:
def train_and_predict_with_xgboost(outcome_idx, time_idx):
    """
    Train an XGBoost model using CIF targets for a specific outcome and time point.
    """
    # Training data
    stacking_inputs_train = np.array([
        final_meta_learner_X_train[key][outcome_idx, time_idx]
        for key in final_meta_learner_X_train.keys()
    ]).T  # Shape: (num_train_samples, num_models)
    
    target_train = cif_ground_truth_test_stacked[outcome_idx, time_idx, :]  # Correct indexing

    # Validation data
    stacking_inputs_val = np.array([
        final_meta_learner_X_val[key][outcome_idx, time_idx]
        for key in final_meta_learner_X_val.keys()
    ]).T  # Shape: (num_val_samples, num_models)

    target_val = cif_ground_truth_val_stacked[outcome_idx, time_idx, :]  # Correct indexing

    # Test data
    stacking_inputs_test = np.array([
        final_meta_learner_X_fin_test[key][outcome_idx, time_idx]
        for key in final_meta_learner_X_fin_test.keys()
    ]).T  # Shape: (num_test_samples, num_models)

    # Prepare DMatrix for XGBoost
    dtrain = xgb.DMatrix(stacking_inputs_train, label=target_train)
    dval = xgb.DMatrix(stacking_inputs_val, label=target_val)
    dtest = xgb.DMatrix(stacking_inputs_test)

    # Configure GPU training
    params = {'objective': 'reg:squarederror',
        'tree_method': 'gpu_hist',
        'max_depth': 3,
        'learning_rate': 0.014837295326564928,
        'subsample': 0.8168005866659258,
        'colsample_bytree': 0.8290129403377126,
        'lambda': 6.7539372305286465,
        'alpha': 0.5018353832953043}

    # Train the model with early stopping
    booster = xgb.train(
        params, dtrain, num_boost_round=500,
        evals=[(dtrain, "train"), (dval, "validation")],
        early_stopping_rounds=20,  # Stop if no improvement for 20 rounds
        verbose_eval=False
    )

    # Predict CIF for test patients
    cif_predictions_test = booster.predict(dtest)
    
    return outcome_idx, time_idx, cif_predictions_test

# Parallel processing for each outcome and time point
results = Parallel(n_jobs=-1)(
    delayed(train_and_predict_with_xgboost)(outcome_idx, time_idx)
    for outcome_idx in range(2)
    for time_idx in range(6)
)

# Assign results to the xgboost_predictions array
xgboost_predictions = np.zeros((2, 6, X_test_transformed.shape[0]))
for outcome_idx, time_idx, cif_predictions in results:
    xgboost_predictions[outcome_idx, time_idx] = cif_predictions

# Verify the shape of the stacked predictions
print("XGBoost Stacking Predictions Shape:", xgboost_predictions.shape)
# Expected output: (2, 6, 316242)


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:55:54] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1730232887822/work/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:55:54] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1730232887822/work/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:55:54] WARNING: /home/conda/feedstock_root/build_ar

XGBoost Stacking Predictions Shape: (2, 6, 40513)


In [160]:
concordance_indices = {}
integrated_brier_scores = {}
neg_log_likelihoods = {}
brier_series = {}
nam_dagostino_results = []  # Initialize as a list

for i in range(0, 2):
    event_interest = i + 1
    cif = pd.DataFrame(xgboost_predictions[i], index=[0, 1, 2, 3, 4, 5])  # Use index correctly
    ev = EvalSurv(1 - cif, durations_fin_test, events_fin_test == event_interest, censor_surv='km')

    # Concordance index
    concordance_indices[f"Event_{event_interest}"] = ev.concordance_td()

    # Brier score series
    brier_series[f"Event_{event_interest}"] = ev.brier_score(TIME_GRID)

    # Integrated Brier score
    integrated_brier_scores[f"Event_{event_interest}"] = ev.integrated_brier_score(TIME_GRID)

    # Negative log-likelihood
    neg_log_likelihoods[f"Event_{event_interest}"] = ev.integrated_nbll(TIME_GRID)

    # Nam and D'Agostino Chi2 statistic for calibration
    for time_idx, time in enumerate(TIME_GRID):
        # Filter data for current time point
        mask = durations_fin_test <= time
        durations_filtered = durations_fin_test[mask]
        events_filtered = events_fin_test[mask]
        
        if len(durations_filtered) == 0:
            print(f"Skipping time {time} for event {event_interest} due to empty data.")
            continue
        
        try:
            # Calculate Nam-D'Agostino Chi² statistic
            chi2_stat, p_value, observed_events, expected_events, n, prob_df = nam_dagostino_chi2(
                df=pd.DataFrame({"durations": durations_filtered, "events": events_filtered}),
                duration_col="durations",
                event_col="events",
                surv=(1 - cif),  # Survival function
                time=time_idx,
                event_focus=event_interest
            )

            # Append results
            nam_dagostino_results.append({
                'Event': event_interest,
                'Year': round(time / 365),
                'Chi2_Stat': chi2_stat,
                'P_Value': p_value,
                'Observed_Events': observed_events.tolist() if isinstance(observed_events, np.ndarray) else observed_events,
                'Expected_Events': expected_events.tolist() if isinstance(expected_events, np.ndarray) else expected_events,
                'Sample_Size': n if isinstance(n, int) else n.tolist()
            })
        except ValueError as e:
            print(f"Error calculating Nam-D'Agostino Chi² at time {time} for event {event_interest}: {e}")

# Display results
print("Concordance Indices:", concordance_indices)
print("Brier Score Series:", brier_series)
print("Integrated Brier Scores:", integrated_brier_scores)
print("Negative Log-Likelihoods:", neg_log_likelihoods)
print("Nam-D'Agostino Results:", nam_dagostino_results)



Error calculating Nam-D'Agostino Chi² at time 0 for event 1: Length of values (40513) does not match length of index (19588)
Error calculating Nam-D'Agostino Chi² at time 0 for event 2: Length of values (40513) does not match length of index (19588)


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event

Concordance Indices: {'Event_1': 0.9792338086166885, 'Event_2': 0.7275737517718853}
Brier Score Series: {'Event_1': 0       0.000000
365     0.034227
730     0.034227
1095    0.034227
1460    0.034227
1825    0.034227
Name: brier_score, dtype: float64, 'Event_2': 0       0.000000
365     0.127521
730     0.127521
1095    0.127521
1460    0.127521
1825    0.127521
Name: brier_score, dtype: float64}
Integrated Brier Scores: {'Event_1': 0.03194480400702308, 'Event_2': 0.11901920846077037}
Negative Log-Likelihoods: {'Event_1': 0.1744737147244062, 'Event_2': 0.37606509292105755}
Nam-D'Agostino Results: [{'Event': 1, 'Year': 1, 'Chi2_Stat': 0.02168567931458386, 'P_Value': 0.9999416396119769, 'Observed_Events': quantile
0    0.00000
1    0.00000
2    0.00000
3    0.00000
4    0.01914
Name: observed_probs, dtype: float64, 'Expected_Events': quantile
0    0.001888
1    0.002650
2    0.003851
3    0.007186
4    0.033434
Name: predicted_probs, dtype: float64, 'Sample_Size': [8104, 8101, 8103, 810

/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [78]:
import numpy as np
import ray.train
import xgboost as xgb
import ray
from ray import tune
from sklearn.model_selection import train_test_split
from pycox.evaluation import EvalSurv
import pandas as pd

# Random seed for reproducibility
RANDOM_SEED = 42

def xgboost_training_wrapper(config, data):
    """
    A Ray Tune-compatible wrapper to train XGBoost models for all outcomes and time points,
    evaluate predictions using concordance index, and report mean concordance index.

    Args:
        config: Dictionary of hyperparameters provided by Ray Tune.
        data: Dictionary containing final_meta_learner_X_train, cif_ground_truth_test_stacked,
              durations, and events.
    """
    final_meta_learner_X_train = data["final_meta_learner_X_train"]
    cif_ground_truth_test_stacked = data["cif_ground_truth_test_stacked"]
    durations = data["durations"]
    events = data["events"]

    # Initialize predictions array (2 outcomes, 6 time points, 316242 patients)
    xgboost_predictions = np.zeros((2, 6, len(durations)))

    # Train models and make predictions for each outcome and time point
    for outcome_idx in range(2):
        for time_idx in range(6):
            # Prepare data
            stacking_inputs = np.array([
                final_meta_learner_X_train[key][outcome_idx, time_idx]
                for key in final_meta_learner_X_train.keys()
            ]).T  # Shape: (316242, 9)

            target = cif_ground_truth_test_stacked[outcome_idx, time_idx, :]  # Shape: (316242,)

            # Split data into training and validation sets
            X_train, X_val, y_train, y_val = train_test_split(
                stacking_inputs, target, test_size=0.2, random_state=RANDOM_SEED
            )

            # Prepare DMatrix for XGBoost
            dtrain = xgb.DMatrix(X_train, label=y_train)
            dval = xgb.DMatrix(X_val, label=y_val)

            # Train XGBoost model
            booster = xgb.train(
                config,
                dtrain,
                num_boost_round=500,
                evals=[(dtrain, "train"), (dval, "validation")],
                early_stopping_rounds=20,
                verbose_eval=False
            )

            # Predict CIF for all patients
            dtest = xgb.DMatrix(stacking_inputs)
            xgboost_predictions[outcome_idx, time_idx] = booster.predict(dtest)

    # Compute concordance index for each event
    concordance_indices = {}
    for i in range(2):
        event_interest = i + 1
        cif = pd.DataFrame(xgboost_predictions[i], index=[0, 1, 2, 3, 4, 5])
        ev = EvalSurv(1 - cif, durations, events == event_interest, censor_surv="km")
        concordance_indices[f"Event_{event_interest}"] = ev.concordance_td()

    # Calculate mean concordance index
    mean_concordance_index = np.mean(list(concordance_indices.values()))

    # Report mean concordance index to Ray Tune
    ray.train.report({'c-index stat': mean_concordance_index})

# Define the search space
search_space = {
    "objective": "reg:squarederror",
    "tree_method": "gpu_hist",  # Use GPU acceleration
    "max_depth": tune.randint(3, 10),  # Maximum depth of trees
    "learning_rate": tune.loguniform(0.01, 0.3),  # Learning rate
    "subsample": tune.uniform(0.5, 1.0),  # Subsampling rate
    "colsample_bytree": tune.uniform(0.5, 1.0),  # Feature subsampling rate
    "lambda": tune.loguniform(1e-4, 10.0),  # L2 regularization term
    "alpha": tune.loguniform(1e-4, 10.0),  # L1 regularization term
}

# Prepare data
data = {
    "final_meta_learner_X_train": final_meta_learner_X_train,
    "cif_ground_truth_test_stacked": cif_ground_truth_test_stacked,
    "durations": durations,
    "events": events,
}

# Run Ray Tune
analysis = tune.run(
    tune.with_parameters(xgboost_training_wrapper, data=data),
    config=search_space,
    resources_per_trial={"cpu": 2, "gpu": 1},  # Adjust based on your resources
    num_samples=50,  # Number of hyperparameter configurations to try
    metric='c-index stat',  # Metric to optimize
    mode="max",  # Maximize the concordance index
    storage_path="/mnt/d/PYDataScience/g3_regress/data/results",  # Directory to store results
)

# Get the best configuration
best_config = analysis.get_best_config(metric="mean_concordance_index", mode="max")
print("Best hyperparameters:", best_config)


2024-11-17 18:31:06,984	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


Trial name,c-index stat
xgboost_training_wrapper_0df1a_00000,0.85425
xgboost_training_wrapper_0df1a_00001,0.852337
xgboost_training_wrapper_0df1a_00002,0.853303
xgboost_training_wrapper_0df1a_00003,0.856744
xgboost_training_wrapper_0df1a_00004,0.855037
xgboost_training_wrapper_0df1a_00005,0.848903
xgboost_training_wrapper_0df1a_00006,0.858138
xgboost_training_wrapper_0df1a_00007,0.848343
xgboost_training_wrapper_0df1a_00008,0.857573
xgboost_training_wrapper_0df1a_00009,0.859644


2024-11-17 18:33:07,651 - INFO - Summary name ray/tune/c-index stat is illegal; using ray/tune/c-index_stat instead.
2024-11-17 18:35:20,666 - INFO - Summary name ray/tune/c-index stat is illegal; using ray/tune/c-index_stat instead.
2024-11-17 18:37:15,012 - INFO - Summary name ray/tune/c-index stat is illegal; using ray/tune/c-index_stat instead.
2024-11-17 18:39:11,679 - INFO - Summary name ray/tune/c-index stat is illegal; using ray/tune/c-index_stat instead.
2024-11-17 18:41:26,716 - INFO - Summary name ray/tune/c-index stat is illegal; using ray/tune/c-index_stat instead.
2024-11-17 18:43:35,691 - INFO - Summary name ray/tune/c-index stat is illegal; using ray/tune/c-index_stat instead.
2024-11-17 18:45:37,053 - INFO - Summary name ray/tune/c-index stat is illegal; using ray/tune/c-index_stat instead.
2024-11-17 18:47:44,061 - INFO - Summary name ray/tune/c-index stat is illegal; using ray/tune/c-index_stat instead.
2024-11-17 18:49:45,263 - INFO - Summary name ray/tune/c-index s

Best hyperparameters: None


In [122]:
import os
import json

# Define the base directory containing the trial folders
base_dir = "/mnt/d/PYDataScience/g3_regress/data/results/xgboost_training_wrapper_2024-11-17_18-31-06"

# Initialize variables to store the max c-index and corresponding config
max_c_index = float("-inf")
best_config = None

# Iterate through all folders and parse the result.json files
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file == "result.json":
            file_path = os.path.join(root, file)
            with open(file_path, "r") as f:
                data = json.load(f)
                if "c-index stat" in data:
                    c_index = data["c-index stat"]
                    # Update max c-index and config if a new max is found
                    if c_index > max_c_index:
                        max_c_index = c_index
                        best_config = data.get("config", None)

# Display the results
max_c_index, best_config


(0.8614533246252081,
 {'objective': 'reg:squarederror',
  'tree_method': 'gpu_hist',
  'max_depth': 3,
  'learning_rate': 0.014837295326564928,
  'subsample': 0.8168005866659258,
  'colsample_bytree': 0.8290129403377126,
  'lambda': 6.7539372305286465,
  'alpha': 0.5018353832953043})

##### 6.3.2 Linear Regression

In [142]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np
from joblib import Parallel, delayed

# Initialize the final stacking predictions array
lineregression_predictions = np.zeros((2, 6, X_test_transformed.shape[0]))  # (2 outcomes, 6 time points, 316242 patients)

def train_and_predict_with_linear_regression(outcome_idx, time_idx):
    """
    Train a Linear Regression model using CIF targets for a specific outcome and time point.
    """
    # Prepare stacking inputs and targets
    # Training data
    stacking_inputs_train = np.array([
        final_meta_learner_X_train[key][outcome_idx, time_idx]
        for key in final_meta_learner_X_train.keys()
    ]).T  # Shape: (num_train_samples, num_models)
    
    target_train = cif_ground_truth_test_stacked[outcome_idx, time_idx, :]  # Correct indexing

    # Validation data
    stacking_inputs_val = np.array([
        final_meta_learner_X_val[key][outcome_idx, time_idx]
        for key in final_meta_learner_X_val.keys()
    ]).T  # Shape: (num_val_samples, num_models)

    target_val = cif_ground_truth_val_stacked[outcome_idx, time_idx, :]  # Correct indexing

    # Test data
    stacking_inputs_test = np.array([
        final_meta_learner_X_fin_test[key][outcome_idx, time_idx]
        for key in final_meta_learner_X_fin_test.keys()
    ]).T  # Shape: (num_test_samples, num_models)

    # Train the Linear Regression model
    model = LinearRegression()
    model.fit(stacking_inputs_train, target_train)

    # Predict CIF for all patients
    cif_predictions = model.predict(stacking_inputs_test)

    return outcome_idx, time_idx, cif_predictions

# Parallel processing for each outcome and time point
results = Parallel(n_jobs=-1)(
    delayed(train_and_predict_with_linear_regression)(outcome_idx, time_idx)
    for outcome_idx in range(2)
    for time_idx in range(6)
)

# Assign results to the lineregression_predictions array
for outcome_idx, time_idx, cif_predictions in results:
    lineregression_predictions[outcome_idx, time_idx] = cif_predictions

# Verify the shape of the stacked predictions
print("Linear Regression Stacking Predictions Shape:", lineregression_predictions.shape)

Linear Regression Stacking Predictions Shape: (2, 6, 40513)


In [161]:
concordance_indices = {}
integrated_brier_scores = {}
neg_log_likelihoods = {}
brier_series = {}
nam_dagostino_results = []  # Initialize as a list

for i in range(0, 2):
    event_interest = i + 1
    cif = pd.DataFrame(lineregression_predictions[i], index=[0, 1, 2, 3, 4, 5])  # Use index correctly
    ev = EvalSurv(1 - cif, durations_fin_test, events_fin_test == event_interest, censor_surv='km')

    # Concordance index
    concordance_indices[f"Event_{event_interest}"] = ev.concordance_td()

    # Brier score series
    brier_series[f"Event_{event_interest}"] = ev.brier_score(TIME_GRID)

    # Integrated Brier score
    integrated_brier_scores[f"Event_{event_interest}"] = ev.integrated_brier_score(TIME_GRID)

    # Negative log-likelihood
    neg_log_likelihoods[f"Event_{event_interest}"] = ev.integrated_nbll(TIME_GRID)

    # Nam and D'Agostino Chi2 statistic for calibration
    for time_idx, time in enumerate(TIME_GRID):
        # Filter data for current time point
        mask = durations_fin_test <= time
        durations_filtered = durations_fin_test[mask]
        events_filtered = events_fin_test[mask]
        
        if len(durations_filtered) == 0:
            print(f"Skipping time {time} for event {event_interest} due to empty data.")
            continue
        
        try:
            # Calculate Nam-D'Agostino Chi² statistic
            chi2_stat, p_value, observed_events, expected_events, n, prob_df = nam_dagostino_chi2(
                df=pd.DataFrame({"durations": durations_filtered, "events": events_filtered}),
                duration_col="durations",
                event_col="events",
                surv=(1 - cif),  # Survival function
                time=time_idx,
                event_focus=event_interest
            )

            # Append results
            nam_dagostino_results.append({
                'Event': event_interest,
                'Year': round(time / 365),
                'Chi2_Stat': chi2_stat,
                'P_Value': p_value,
                'Observed_Events': observed_events.tolist() if isinstance(observed_events, np.ndarray) else observed_events,
                'Expected_Events': expected_events.tolist() if isinstance(expected_events, np.ndarray) else expected_events,
                'Sample_Size': n if isinstance(n, int) else n.tolist()
            })
        except ValueError as e:
            print(f"Error calculating Nam-D'Agostino Chi² at time {time} for event {event_interest}: {e}")

# Display results
print("Concordance Indices:", concordance_indices)
print("Brier Score Series:", brier_series)
print("Integrated Brier Scores:", integrated_brier_scores)
print("Negative Log-Likelihoods:", neg_log_likelihoods)
print("Nam-D'Agostino Results:", nam_dagostino_results)



Error calculating Nam-D'Agostino Chi² at time 0 for event 1: Length of values (40513) does not match length of index (19588)
Error calculating Nam-D'Agostino Chi² at time 0 for event 2: Length of values (40513) does not match length of index (19588)


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event

Concordance Indices: {'Event_1': 0.979147976082038, 'Event_2': 0.7546631563758323}
Brier Score Series: {'Event_1': 0       0.00000
365     0.03744
730     0.03744
1095    0.03744
1460    0.03744
1825    0.03744
Name: brier_score, dtype: float64, 'Event_2': 0       0.000000
365     0.116634
730     0.116634
1095    0.116634
1460    0.116634
1825    0.116634
Name: brier_score, dtype: float64}
Integrated Brier Scores: {'Event_1': 0.034943942825916126, 'Event_2': 0.1088581410271849}
Negative Log-Likelihoods: {'Event_1': 0.18104063587078995, 'Event_2': 0.3927945233629699}
Nam-D'Agostino Results: [{'Event': 1, 'Year': 1, 'Chi2_Stat': 0.020654911205768265, 'P_Value': 0.9999470375752154, 'Observed_Events': quantile
0    0.000000
1    0.000000
2    0.000000
3    0.000000
4    0.019646
Name: observed_probs, dtype: float64, 'Expected_Events': quantile
0    0.001286
1    0.002432
2    0.003715
3    0.006436
4    0.035073
Name: predicted_probs, dtype: float64, 'Sample_Size': [8103, 8102, 8103, 8102

/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(


##### 6.3.3 RandomForest regression

In [125]:
from sklearn.ensemble import RandomForestRegressor
from joblib import Parallel, delayed
import numpy as np

def train_and_predict_with_rf(outcome_idx, time_idx):
    """
    Train a Random Forest model using CIF targets for a specific outcome and time point.
    Use pre-prepared validation and test predictions.
    """
    # Prepare training data
    stacking_inputs_train = np.array([
        final_meta_learner_X_train[key][outcome_idx, time_idx]
        for key in final_meta_learner_X_train.keys()
    ]).T  # Shape: (num_train_samples, num_models)
    
    target_train = cif_ground_truth_test_stacked[outcome_idx, time_idx, :]  # Correct indexing

    # Validation data
    stacking_inputs_val = np.array([
        final_meta_learner_X_val[key][outcome_idx, time_idx]
        for key in final_meta_learner_X_val.keys()
    ]).T  # Shape: (num_val_samples, num_models)

    target_val = cif_ground_truth_val_stacked[outcome_idx, time_idx, :]  # Correct indexing

    # Test data
    stacking_inputs_test = np.array([
        final_meta_learner_X_fin_test[key][outcome_idx, time_idx]
        for key in final_meta_learner_X_fin_test.keys()
    ]).T  # Shape: (num_test_samples, num_models)

    # Train the Random Forest Regressor
    model = RandomForestRegressor(
        n_estimators=168,
        max_depth=3,
        min_samples_split=3,
        min_samples_leaf=4,
        random_state=RANDOM_SEED,
        max_features='sqrt',
        n_jobs=-1  # Use all available cores,
        
    )
    model.fit(stacking_inputs_train, target_train, )

    # Predict CIF for all patients in the test set
    cif_predictions = model.predict(stacking_inputs_test)

    return outcome_idx, time_idx, cif_predictions

# Parallel processing for each outcome and time point
rf_predictions = np.zeros((2, 6, X_test_transformed.shape[0]))  # Initialize prediction array
results = Parallel(n_jobs=-1)(
    delayed(train_and_predict_with_rf)(outcome_idx, time_idx)
    for outcome_idx in range(2)
    for time_idx in range(6)
)

# Assign results to the rf_predictions array
for outcome_idx, time_idx, cif_predictions in results:
    rf_predictions[outcome_idx, time_idx] = cif_predictions

# Verify the shape of the stacked predictions
print("Random Forest Stacking Predictions Shape:", rf_predictions.shape)
# Expected output: (2, 6, num_test_samples)


Random Forest Stacking Predictions Shape: (2, 6, 40513)


In [162]:
concordance_indices = {}
integrated_brier_scores = {}
neg_log_likelihoods = {}
brier_series = {}
nam_dagostino_results = []  # Initialize as a list

for i in range(0, 2):
    event_interest = i + 1
    cif = pd.DataFrame(rf_predictions[i], index=[0, 1, 2, 3, 4, 5])  # Use index correctly
    ev = EvalSurv(1 - cif, durations_fin_test, events_fin_test == event_interest, censor_surv='km')

    # Concordance index
    concordance_indices[f"Event_{event_interest}"] = ev.concordance_td()

    # Brier score series
    brier_series[f"Event_{event_interest}"] = ev.brier_score(TIME_GRID)

    # Integrated Brier score
    integrated_brier_scores[f"Event_{event_interest}"] = ev.integrated_brier_score(TIME_GRID)

    # Negative log-likelihood
    neg_log_likelihoods[f"Event_{event_interest}"] = ev.integrated_nbll(TIME_GRID)

    # Nam and D'Agostino Chi2 statistic for calibration
    for time_idx, time in enumerate(TIME_GRID):
        # Filter data for current time point
        mask = durations_fin_test <= time
        durations_filtered = durations_fin_test[mask]
        events_filtered = events_fin_test[mask]
        
        if len(durations_filtered) == 0:
            print(f"Skipping time {time} for event {event_interest} due to empty data.")
            continue
        
        try:
            # Calculate Nam-D'Agostino Chi² statistic
            chi2_stat, p_value, observed_events, expected_events, n, prob_df = nam_dagostino_chi2(
                df=pd.DataFrame({"durations": durations_filtered, "events": events_filtered}),
                duration_col="durations",
                event_col="events",
                surv=(1 - cif),  # Survival function
                time=time_idx,
                event_focus=event_interest
            )

            # Append results
            nam_dagostino_results.append({
                'Event': event_interest,
                'Year': round(time / 365),
                'Chi2_Stat': chi2_stat,
                'P_Value': p_value,
                'Observed_Events': observed_events.tolist() if isinstance(observed_events, np.ndarray) else observed_events,
                'Expected_Events': expected_events.tolist() if isinstance(expected_events, np.ndarray) else expected_events,
                'Sample_Size': n if isinstance(n, int) else n.tolist()
            })
        except ValueError as e:
            print(f"Error calculating Nam-D'Agostino Chi² at time {time} for event {event_interest}: {e}")

# Display results
print("Concordance Indices:", concordance_indices)
print("Brier Score Series:", brier_series)
print("Integrated Brier Scores:", integrated_brier_scores)
print("Negative Log-Likelihoods:", neg_log_likelihoods)
print("Nam-D'Agostino Results:", nam_dagostino_results)



Error calculating Nam-D'Agostino Chi² at time 0 for event 1: Length of values (40513) does not match length of index (19588)
Error calculating Nam-D'Agostino Chi² at time 0 for event 2: Length of values (40513) does not match length of index (19588)


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event

Concordance Indices: {'Event_1': 0.9796230921738424, 'Event_2': 0.7554581494270163}
Brier Score Series: {'Event_1': 0       0.000000
365     0.053315
730     0.053315
1095    0.053315
1460    0.053315
1825    0.053315
Name: brier_score, dtype: float64, 'Event_2': 0       0.000000
365     0.099329
730     0.099329
1095    0.099329
1460    0.099329
1825    0.099329
Name: brier_score, dtype: float64}
Integrated Brier Scores: {'Event_1': 0.0497605494173283, 'Event_2': 0.0927068468793533}
Negative Log-Likelihoods: {'Event_1': 0.23971583618508627, 'Event_2': 0.3280149880111438}
Nam-D'Agostino Results: [{'Event': 1, 'Year': 1, 'Chi2_Stat': 0.02078836491212447, 'P_Value': 0.9999463533563095, 'Observed_Events': quantile
0    0.000000
1    0.000000
2    0.000000
3    0.000000
4    0.020235
Name: observed_probs, dtype: float64, 'Expected_Events': quantile
0    0.003513
1    0.003532
2    0.003596
3    0.005635
4    0.032308
Name: predicted_probs, dtype: float64, 'Sample_Size': [12648, 3869, 7812,

/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(


In [118]:
def random_forest_training_wrapper(config, data):
    """
    A Ray Tune-compatible wrapper to train Random Forest models for all outcomes and time points,
    evaluate predictions using concordance index, and report mean concordance index.

    Args:
        config: Dictionary of hyperparameters provided by Ray Tune.
        data: Dictionary containing final_meta_learner_X_train, cif_ground_truth_test_stacked,
              durations, and events.
    """
    final_meta_learner_X_train = data["final_meta_learner_X_train"]
    final_meta_learner_X_val = data["final_meta_learner_X_val"]
    cif_ground_truth_test_stacked = data["cif_ground_truth_test_stacked"]
    cif_ground_truth_val_stacked = data["cif_ground_truth_val_stacked"]
    durations_val = data["durations_val"]
    events_val = data["events_val"]

    # Initialize predictions array (2 outcomes, 6 time points, number of patients)
    rf_predictions = np.zeros((2, 6, X_fin_val.shape[0]))

    # Train models and make predictions for each outcome and time point
    for outcome_idx in range(2):
        for time_idx in range(6):
            # Prepare training data
            stacking_inputs_train = np.array([
                final_meta_learner_X_train[key][outcome_idx, time_idx]
                for key in final_meta_learner_X_train.keys()
            ]).T  # Shape: (num_train_samples, num_models)
            
            target_train = cif_ground_truth_test_stacked[outcome_idx, time_idx, :]  # Correct indexing

            # Validation data
            stacking_inputs_val = np.array([
                final_meta_learner_X_val[key][outcome_idx, time_idx]
                for key in final_meta_learner_X_val.keys()
            ]).T  # Shape: (num_val_samples, num_models)

            target_val = cif_ground_truth_val_stacked[outcome_idx, time_idx, :]  # Correct indexing

            # Train Random Forest model
            model = RandomForestRegressor(
                n_estimators=config["n_estimators"],
                max_depth=config["max_depth"],
                min_samples_split=config["min_samples_split"],
                min_samples_leaf=config["min_samples_leaf"],
                max_features=config["max_features"],
                random_state=RANDOM_SEED,
                n_jobs=-1
            )
            model.fit(stacking_inputs_train , target_train)

            # Predict CIF for all patients
            rf_predictions[outcome_idx, time_idx] = model.predict(stacking_inputs_val)

    # Compute concordance index for each event
    concordance_indices = {}
    for i in range(2):
        event_interest = i + 1
        cif = pd.DataFrame(rf_predictions[i], index=[0, 1, 2, 3, 4, 5])
        ev = EvalSurv(1 - cif, durations_val, events_val == event_interest, censor_surv="km")
        concordance_indices[f"Event_{event_interest}"] = ev.concordance_td()

    # Calculate mean concordance index
    mean_concordance_index = np.mean(list(concordance_indices.values()))

    # Report mean concordance index to Ray Tune
    ray.train.report({'c-index_stat': mean_concordance_index})

# Define the search space
search_space = {
    "n_estimators": tune.randint(50, 500),  # Number of trees in the forest
    "max_depth": tune.randint(3, 20),  # Maximum depth of the tree
    "min_samples_split": tune.randint(2, 10),  # Minimum number of samples required to split an internal node
    "min_samples_leaf": tune.randint(1, 10),  # Minimum number of samples required to be at a leaf node
    "max_features": tune.choice(["sqrt", "log2"])  # Number of features to consider when looking for the best split
}

# Prepare data
data = {
    "final_meta_learner_X_train": final_meta_learner_X_train,
    "final_meta_learner_X_val": final_meta_learner_X_val,
    "cif_ground_truth_test_stacked": cif_ground_truth_test_stacked,
    "cif_ground_truth_val_stacked": cif_ground_truth_val_stacked,
    "durations_val": durations_val,
    "events_val": events_val,
}

ray.shutdown()
ray.init()
# Run Ray Tune
analysis = tune.run(
    tune.with_parameters(random_forest_training_wrapper, data=data),
    config=search_space,
    resources_per_trial={"cpu": 20, "gpu": 0},  # Random Forest does not require GPU
    num_samples=50,  # Number of hyperparameter configurations to try
    metric="c-index_stat",  # Metric to optimize
    mode="max",  # Maximize the concordance index
    storage_path="/mnt/d/PYDataScience/g3_regress/data/results",  # Directory to store results
    verbose=2
)

# Get the best configuration
best_config = analysis.get_best_config(metric="c-index stat", mode="max")
print("Best hyperparameters:", best_config)

ray.shutdown()

2024-11-19 07:06:14,475	INFO worker.py:1807 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2024-11-19 07:06:15,356	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(raylet) Warning: The actor ImplicitFunc is very large (16 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


Trial name,c-index_stat
random_forest_training_wrapper_b6411_00000,0.857171
random_forest_training_wrapper_b6411_00001,0.859153
random_forest_training_wrapper_b6411_00002,0.861421
random_forest_training_wrapper_b6411_00003,0.849436
random_forest_training_wrapper_b6411_00004,0.853591
random_forest_training_wrapper_b6411_00005,0.860848
random_forest_training_wrapper_b6411_00006,0.857037
random_forest_training_wrapper_b6411_00007,0.860378
random_forest_training_wrapper_b6411_00008,0.861465
random_forest_training_wrapper_b6411_00009,0.857075


2024-11-19 08:39:23,246	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/mnt/d/PYDataScience/g3_regress/data/results/random_forest_training_wrapper_2024-11-19_07-06-15' in 1.2829s.
2024-11-19 08:39:23,252	INFO tune.py:1041 -- Total run time: 5587.90 seconds (5586.48 seconds for the tuning loop).
2024-11-19 08:39:23,467	WARNING experiment_analysis.py:558 -- Could not find best trial. Did you pass the correct `metric` parameter?


Best hyperparameters: None


In [124]:
import os
import json

# Define the base directory containing the trial folders
base_dir = "/mnt/d/PYDataScience/g3_regress/data/results/random_forest_training_wrapper_2024-11-19_07-06-15"

# Initialize variables to store the max c-index and corresponding config
max_c_index = float("-inf")
best_config = None

# Iterate through all folders and parse the result.json files
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file == "result.json":
            file_path = os.path.join(root, file)
            with open(file_path, "r") as f:
                data = json.load(f)
                if "c-index_stat" in data:
                    c_index = data["c-index_stat"]
                    # Update max c-index and config if a new max is found
                    if c_index > max_c_index:
                        max_c_index = c_index
                        best_config = data.get("config", None)

# Display the results
max_c_index, best_config


(0.8617197160068155,
 {'n_estimators': 168,
  'max_depth': 3,
  'min_samples_split': 3,
  'min_samples_leaf': 4,
  'max_features': 'sqrt'})